In [1]:
import pickle
import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

print('Done importing')

Using TensorFlow backend.


Done importing


In [2]:
# Load data from file
training_data_file = './train_aug.p'
with open(training_data_file, mode='rb') as f:
  train = pickle.load(f)

X_train, y_train = train['features'], train['labels']
print('Done data loading')

Done data loading


In [ ]:
model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Convolution2D(6, 5, 5, activation='relu'))
model.add(MaxPooling2D())
model.add(Convolution2D(6, 5, 5, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120)) 
model.add(Dense(84)) 
model.add(Dense(1)) 


model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=7)
        
model.save('model.h5')

Train on 12857 samples, validate on 3215 samples
Epoch 1/7
12857/12857 [==============================] - 49s - loss: 2.6208 - val_loss: 0.0130
Epoch 2/7
12857/12857 [==============================] - 44s - loss: 0.0131 - val_loss: 0.0116oss: 0.
Epoch 3/7
12857/12857 [==============================] - 44s - loss: 0.0118 - val_loss: 0.0107s: 0 - ETA: 8s - loss: 0.011  - ETA: 3 - ETA: 2s - loss: 0.0 - ET
Epoch 4/7
12857/12857 [==============================] - 44s - loss: 0.0108 - val_loss: 0.0101s - ETA: 2s - loss: 0.010 - E
Epoch 5/7
12857/12857 [==============================] - 44s - loss: 0.0100 - val_loss: 0.0096
Epoch 6/7
12857/12857 [==============================] - 44s - loss: 0.0093 - val_loss: 0.0093
Epoch 7/7
 2208/12857 [====>.........................] - ETA: 33s - loss: 0.0076 ETA: 37s - lo - E